Import and initialize the Earth Engine Python API.

In [ ]:
import ee
ee.Initialize()

Import a customized version of IPyLeaflet. The customized version is defined in the file ipyleafletee.py, which is stored in the same folder as this notebook. 

In [ ]:
import ipyleafletee

Display the default map. The basemap is based on NOAA's [ETOPO1](https://explorer.earthengine.google.com/#detail/NOAA%2FNGDC%2FETOPO1) elevation dataset.

In [ ]:
map1 = ipyleafletee.Map(zoom=2)
map1

Examine the map object, to see what it can do.

In [ ]:
# Generate a list of methods and properties, suppressing private methods that start with and underscore.
method_and_property_list = ', '.join([prop for prop in dir(map1) if not prop.startswith('_')])
print(method_and_property_list)

As we can see, this map object can do a lot. For example, you can set and get the center of the map.

In [ ]:
# Get the current map center and zoom level.
print('center coords = {0}'.format(map1.center))
print('zoom = {0}'.format(map1.zoom))

In [ ]:
# Set the current map center and zoom level (to the San Francisco Bay Area)
map1.center = [37.9, -122.0]
map1.zoom =6

Create a second map, using a custom basemap.

In [ ]:
def getTileLayerUrl(map_id):
  template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return template.format(**map_id)

custom_basemap_mapid = ee.Image('NOAA/NGDC/ETOPO1').select('bedrock').gt(0).getMapId()

map2 = ipyleafletee.Map(
  zoom=2,
  center=[37.9, -122.0],
  default_tiles=ipyleafletee.TileLayerEE(
    url=getTileLayerUrl(custom_basemap_mapid)
  )
)
map2

Add an overlay layer to the map.

In [ ]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
occurrence = gsw.select('occurrence')
map_id = occurrence.visualize(
  min=0,
  max=100,
  palette= ['red', 'blue']
).updateMask(occurrence.divide(100)).getMapId()

jrc_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
tl_jrc = ipyleafletee.TileLayer(url=jrc_tile_url)
map2.add_layer(tl_jrc)